In [1]:
import os
import json
from web3 import Web3, HTTPProvider

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from utils.subgraph_utils.convex_community import get_curve_fees
from utils.etherscan_utils import get_txes_between_blocks

In [2]:
ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
web3 = Web3(HTTPProvider(f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}"))

In [4]:
pool_addr = "0xEB16Ae0052ed37f479f7fe63849198Df1765a733"
lp_token_addr = "0x02d341CcB60fAaf662bC0554d13778015d1b285C"
dai_addr = "0x6B175474E89094C44Da98b954EedeAC495271d0F"
susd_addr = "0x57Ab1ec28D129707052df4dF418D58a2D46d5f51"

In [5]:
admin_2 = "0xA464e6DCda8AC41e03616F95f4BC98a13b8922Dc"
admin = "0xeCb456EA5365865EbAb8a2661B0c503410e9B347"

In [6]:
pool_addr.lower()

'0xeb16ae0052ed37f479f7fe63849198df1765a733'

In [11]:
with open("./susd.json", 'r') as f:
    abi = json.load(f)
susd = web3.eth.contract(address=susd_addr, abi=abi)

with open("./dai.json", 'r') as f:
    abi = json.load(f)
dai = web3.eth.contract(address=dai_addr, abi=abi)

In [7]:
subgraph_tripool_fees = get_curve_fees(pool_token_addr=lp_token_addr)
blocks = list([int(i) for i in subgraph_tripool_fees.block.values])
subgraph_block_ranges = list(zip(blocks, blocks[1:]))
subgraph_tripool_fees.index = subgraph_tripool_fees.block

In [9]:
ecb_txes = get_txes_between_blocks(address=admin.lower(), from_block=blocks[0], to_block=blocks[-1])
ecb_tx_blocks = [int(i['blockNumber']) for i in ecb_txes]
admin_2_txes = get_txes_between_blocks(address=admin_2.lower(), from_block=blocks[0], to_block=blocks[-1])
admin_2_tx_blocks = [int(i['blockNumber']) for i in admin_2_txes]

tx_blocks = list(set(ecb_tx_blocks) - set(admin_2_tx_blocks))
tx_blocks = sorted(tx_blocks)
tx_blocks

[12730716,
 12730729,
 12731126,
 12731135,
 12731141,
 12731150,
 12731183,
 12737439,
 12737458,
 12737484,
 12737493,
 12737510,
 12737516,
 12737526,
 12768210,
 12776169,
 12776192,
 12776212,
 12776222,
 12776234,
 12776246,
 12776269,
 12808930,
 12823285,
 12823301,
 12823319,
 12823331,
 12823347,
 12823354,
 12823388,
 12828194,
 12828214,
 12871333,
 12871347,
 12871427,
 12871442,
 12871457,
 12871469,
 12871512,
 12916292,
 12916437,
 12916447,
 12916458,
 12916468,
 12916476,
 12916485,
 12916494,
 12916501,
 12916510,
 12916520,
 12916530,
 12916539,
 12916552,
 12916566,
 12916578,
 12916586,
 12916594,
 12916603,
 12916614,
 12916625,
 12916638,
 12916656,
 12916665,
 12916678,
 12916686,
 12916698,
 12916710,
 12916722,
 12916731,
 12916759,
 12916780,
 12916791,
 12916803,
 12916819,
 12916831,
 12916841,
 12916848,
 12916910,
 12916943,
 12916965,
 12916974,
 12917005,
 12917023,
 12953882,
 12953891,
 12954443,
 12954457,
 12954467,
 12954481,
 12954514,
 12973135,

In [12]:
fees_for_range = []
for block in tx_blocks:

    block_start = block - 1
    block_end = block + 1

    print(f"Chain checkoooor-ing blocks: {block_start}:{block_end}")

    susd_transfer_filter = susd.events.Transfer.createFilter(
        fromBlock=block_start,
        toBlock=block_end
    )
    all_transfers = susd_transfer_filter.get_all_entries()
    susd_fees = 0
    for transfer in all_transfers:

        to_address = transfer['args']['to']
        from_address = transfer['args']['from']
        value = transfer['args']['value']
        block = transfer['blockNumber']

        if (
                from_address == pool_addr and
                to_address in [admin_2, admin]
        ):
            print(f"susd fees: {value * 1e-6}")
            susd_fees += value * 1e-18

    dai_transfer_filter = dai.events.Transfer.createFilter(
        fromBlock=block_start,
        toBlock=block_end
    )
    all_transfers = dai_transfer_filter.get_all_entries()
    dai_fees = 0
    for transfer in all_transfers:

        to_address = transfer['args']['to']
        from_address = transfer['args']['from']
        value = transfer['args']['value']
        block = transfer['blockNumber']

        if (
                from_address == pool_addr and
                to_address in [admin_2, admin]
        ):
            print(f"dai fees: {value * 1e-18}")
            dai_fees += value * 1e-18

    dict_fees = {
        'block_start': block_start,
        'block_end': block_end,
        'susd_fees': susd_fees,
        'dai_fees': dai_fees,
    }
    fees_for_range.append(dict_fees)

df_fees = pd.DataFrame(fees_for_range)
df_fees

Chain checkoooor-ing blocks: 12730715:12730717
Chain checkoooor-ing blocks: 12730728:12730730
Chain checkoooor-ing blocks: 12731125:12731127
Chain checkoooor-ing blocks: 12731134:12731136
Chain checkoooor-ing blocks: 12731140:12731142
Chain checkoooor-ing blocks: 12731149:12731151
Chain checkoooor-ing blocks: 12731182:12731184
Chain checkoooor-ing blocks: 12737438:12737440
Chain checkoooor-ing blocks: 12737457:12737459
Chain checkoooor-ing blocks: 12737483:12737485
Chain checkoooor-ing blocks: 12737492:12737494
Chain checkoooor-ing blocks: 12737509:12737511
Chain checkoooor-ing blocks: 12737515:12737517
Chain checkoooor-ing blocks: 12737525:12737527
Chain checkoooor-ing blocks: 12768209:12768211
Chain checkoooor-ing blocks: 12776168:12776170
Chain checkoooor-ing blocks: 12776191:12776193
Chain checkoooor-ing blocks: 12776211:12776213
Chain checkoooor-ing blocks: 12776221:12776223
Chain checkoooor-ing blocks: 12776233:12776235
Chain checkoooor-ing blocks: 12776245:12776247
Chain checkoo

,block_start,block_end,usdc_fees,dai_fees
0,12730715,12730717,0,0
1,12730728,12730730,0,0
2,12731125,12731127,0,0
3,12731134,12731136,0,0
4,12731140,12731142,0,0
...,...,...,...,...
268,13717485,13717487,0,0
269,13717489,13717491,0,0
270,13717495,13717497,0,0
271,13717518,13717520,0,0


In [ ]:
df_fees['total_fees'] = df_fees.susd_fees + df_fees.dai_fees
df_fees.index = df_fees.block_end
df_fees = df_fees.where(df_fees.total_fees > 0).dropna()
df_fees

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20, 8))

(df_fees['total_fees']*2).plot(
    ax=ax,
    color='red',
    kind='bar',
    label='Admin fee claims x 2',
    width=0.2,
    position=1,
)

ax.set_ylabel("Pool Revenue [$]", fontsize=25)
ax.set_xlabel("Block Number", fontsize=25)
ax.tick_params(axis='both', which='major', labelsize=25, colors='black')
ax.yaxis.get_major_formatter().set_scientific(False)
ax.yaxis.get_major_formatter().set_useOffset(False)
ax.legend(fontsize=25);

In [ ]:
aggregated_claims = []
blocks = []
for (block_start, block_end) in subgraph_block_ranges:

    aggregated_claims.append(
        df_fees.total_fees.where(
            (block_start < df_fees.index) & (df_fees.index < block_end)
        ).sum()
    )
    blocks.append(block_end)

df_aggregated_claims = pd.DataFrame(data=aggregated_claims, index=blocks, columns=['claimed'])
df_aggregated_claims

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(20, 8))

(df_aggregated_claims['claimed']*2).plot(
    ax=ax,
    color='red',
    kind='bar',
    label='Admin fee claims x 2',
    width=0.2,
    position=1,
)
subgraph_tripool_fees['fees'].plot(
    ax=ax,
    color='blue',
    kind='bar',
    label='Subgraph revenue',
    width=0.2,
    position=2,
)

ax.set_ylabel("Pool Revenue [$]", fontsize=25)
ax.set_xlabel("Block Number", fontsize=25)
ax.tick_params(axis='both', which='major', labelsize=25, colors='black')
ax.yaxis.get_major_formatter().set_scientific(False)
ax.yaxis.get_major_formatter().set_useOffset(False)
ax.legend(fontsize=25);